# (목적)Gradient를 확인한다
## 전체 Gradient에 3을 곱한것과 parameter 별로 gradient에 3을 곱한 것이 차이가 나는지 확인

In [41]:
import torch
import torch.nn as nn

import torch.nn.functional as F

# 예제 데이터 생성
X = torch.randn(100, 2)
y = torch.randint(0, 2, (100,))

# 모델 정의
class ClassificationModel(nn.Module):
    def __init__(self):
        super(ClassificationModel, self).__init__()
        self.fc1 = nn.Linear(2, 2)
        self.fc2 = nn.Linear(2, 2)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x

model = ClassificationModel()

# Cross Entropy Loss와 KL Divergence Loss 정의
cross_entropy_loss = nn.CrossEntropyLoss()
kl_divergence_loss = nn.KLDivLoss(reduction='batchmean')

# Forward Pass
outputs = model(X)

# Cross Entropy Loss 계산
ce_loss = F.cross_entropy(outputs, y)

# KL Divergence Loss 계산
p = torch.tensor([0.1, 0.2])  # 예시 분포
kl_loss = kl_divergence_loss(torch.log(outputs), p)
# kl_loss = ce_loss

# Gradient 계산 (torch.autograd.grad 사용)
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
kl_gradient = torch.autograd.grad(kl_loss, model.parameters(), retain_graph=True)

# Gradient 확인
print("Cross Entropy Loss Gradient:")
for grad_param in ce_gradient:
    print(grad_param)

# print("\nKL Divergence Loss Gradient:")
# for grad_param in kl_gradient:
#     print(grad_param)

Cross Entropy Loss Gradient:
tensor([[-0.0303,  0.0916],
        [-0.0042,  0.0128]])
tensor([0.3576, 0.0499])
tensor([[-0.0640, -0.0393],
        [ 0.0640,  0.0393]])
tensor([-0.2848,  0.2848])


In [42]:
ce_loss *= 2
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)

# Gradient 확인
print("Cross Entropy Loss Gradient:")
for grad_param in ce_gradient:
    print(grad_param)

Cross Entropy Loss Gradient:
tensor([[-0.0607,  0.1832],
        [-0.0085,  0.0256]])
tensor([0.7153, 0.0998])
tensor([[-0.1280, -0.0787],
        [ 0.1280,  0.0787]])
tensor([-0.5695,  0.5695])


In [ ]:
kl_loss

In [ ]:
total_loss = 1 * ce_loss +  2* kl_loss
total_loss
total_gradient = torch.autograd.grad(total_loss, model.parameters(), retain_graph=True)
print(total_gradient)

In [ ]:
names_grads_copy = {}

for param_name, ce_grad, kl_grad in zip(model.parameters(), ce_gradient, kl_gradient):
    
    #kl_grad = None
    
    if not kl_grad == None:
        names_grads_copy[param_name] = torch.tensor(1) * ce_grad + torch.tensor(2) *  kl_grad
    else:
        names_grads_copy[param_name] = torch.tensor(1) * ce_grad 
    
for key, value in names_grads_copy.items():
    print(value)

## 1. Gradient를 추가적으로 구해도 retain_graph=True를 설정하면, 값이 변하지 않는다

In [ ]:
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(grad_param)

# 2. 전체 gradient에 3을 곱한다
## 1) 안된다

In [ ]:
ce_gradient =  torch.tensor(3) * torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(grad_param)

## 2) 덧셈 조차 하면 안된다

In [ ]:
total_gradient = ce_gradient + kl_gradient
total_gradient

## 3) loss에 3을 곱해줘야 내가 원하는 값을 얻을 수 있다

In [ ]:
ce_gradient =  torch.autograd.grad(torch.tensor(3) * ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(grad_param)

# 3. 각 parameter 별로 3을 곱해준다

In [ ]:
# Gradient 확인
print("Cross Entropy Loss Gradient:")
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(torch.tensor(3) * grad_param)